In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint



2024-09-30 13:16:25.499488: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 13:16:25.512087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 13:16:25.525211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 13:16:25.529031: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 13:16:25.539109: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Path to your image dataset
#data_dir = '/home/gcekcse/Documents/ML_Project_hk/aptos2019-blindness-detection/G1/G1_images'
#labels_file = '/home/gcekcse/Documents/ML_Project_hk/aptos2019-blindness-detection/G1/G1.csv'
data_dir = 'C:/Users/heman/Documents/ML_project/aptos2019-blindness-detection/G1/G1_images'
labels_file = 'C:/Users/heman/Documents/ML_project/aptos2019-blindness-detection/G1/G1.csv'


# Load labels
df = pd.read_csv(labels_file)



In [5]:
# Count the number of samples for each class
class_counts = df['diagnosis'].value_counts().reset_index()

# Rename the columns for better understanding
class_counts.columns = ['Class', 'Number of Samples']

# Display the tabular view
print(class_counts)

   Class  Number of Samples
0      0               1805
1      2                999
2      1                370
3      4                295
4      3                193


In [6]:
# Split data into training and testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display sample data
print(train_df.head())

           id_code  diagnosis
2181  996f9bba4ef0          0
570   286e9981dd9b          0
3389  eae70f527755          0
343   19244004583f          3
1801  7f0ffeb0a333          0


In [5]:
'''import pandas as pd
from sklearn.model_selection import train_test_split

# Paths to your dataset
train_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/train_images/train_images/'
test_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/test_images/test_images/'
val_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/val_images/val_images/'
csv_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive'

# Path to CSV files
train_csv = f'{csv_dir}/train_1.csv'
test_csv = f'{csv_dir}/test.csv'
valid_csv = f'{csv_dir}/valid.csv'

# Load the training labels
df = pd.read_csv(train_csv)

# Count the number of samples for each class
class_counts = df['diagnosis'].value_counts().reset_index()

# Split data into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display sample data from the training set
print(train_df.head())

# Rename the columns for better understanding
class_counts.columns = ['Class', 'Number of Samples']

# Display the tabular view of class counts
print(class_counts)
'''

"import pandas as pd\nfrom sklearn.model_selection import train_test_split\n\n# Paths to your dataset\ntrain_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/train_images/train_images/'\ntest_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/test_images/test_images/'\nval_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive/val_images/val_images/'\ncsv_dir = 'C:/Users/heman/Documents/ML_project/DATA2/archive'\n\n# Path to CSV files\ntrain_csv = f'{csv_dir}/train_1.csv'\ntest_csv = f'{csv_dir}/test.csv'\nvalid_csv = f'{csv_dir}/valid.csv'\n\n# Load the training labels\ndf = pd.read_csv(train_csv)\n\n# Count the number of samples for each class\nclass_counts = df['diagnosis'].value_counts().reset_index()\n\n# Split data into training and testing sets (80% train, 20% test)\ntrain_df, test_df = train_test_split(df, test_size=0.2, random_state=42)\n\n# Display sample data from the training set\nprint(train_df.head())\n\n# Rename the columns for better understanding\nclass

In [5]:

IMG_SIZE = 224  # Resize images to 224x224
#Uncomment the below code when it's run first time

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image at {image_path}")
        return None  # Skip images that are not loaded
    else:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0  # Normalize
        return img

def preprocess_data(dataframe, data_dir):
    X = []
    y = []
    for idx, row in dataframe.iterrows():
        img_path = os.path.join(data_dir, row['id_code'] + ".png")  # Assuming .png extension
        img = preprocess_image(img_path)
        
        if img is not None:  # Proceed only if the image was loaded
            X.append(img)
            y.append(row['diagnosis'])  # Assuming 'diagnosis' contains the target label
            
        if idx % 100 == 0:  # Log progress every 100 images
            print(f"Processed {idx + 1}/{len(dataframe)} images.")
    
    return np.array(X), np.array(y)

# Run preprocessing with logging
print("Preprocessing training data...")
X_train, y_train = preprocess_data(train_df, data_dir)

print("Preprocessing test data...")
X_test, y_test = preprocess_data(test_df, data_dir)

print("Preprocessing completed.")

# Save training data
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

# Save test data
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

Preprocessing training data...


NameError: name 'train_df' is not defined

In [3]:
#Loading Data

X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')


In [6]:
# Load ResNet50 base model (pre-trained on ImageNet)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 DR severity levels
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Show the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 74,971,013
Trainable params: 51,383,301
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Define the checkpoint callback to overwrite the saved model after each epoch
checkpoint_callback = ModelCheckpoint(
    filepath=r'C:/Users/heman/Documents/ML_project/models/model.keras',  # Constant file path
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

# Define early stopping callback to monitor validation loss
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop after 5 epochs with no improvement
    restore_best_weights=True
)

Num GPUs Available:  1


In [ ]:
# Fit the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=80,
    callbacks=[checkpoint_callback, early_stopping]
)


Epoch 1/80


In [ ]:
#!tensorboard --logdir=logs/fit


In [4]:
# Load the model from the specified file path
model = load_model(r'/home/gcekcse/Documents/ML_Project_hk/models/model.h5')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

# Convert your input to a tensor
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test_tensor, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Classification report with zero_division set to handle cases with no predicted samples
y_pred = np.argmax(model.predict(X_test_tensor), axis=1)
print(classification_report(y_test, y_pred, zero_division=1))

# ROC-AUC score (use predicted probabilities instead of class labels)
y_pred_prob = model.predict(X_test_tensor)  # Get the predicted probabilities
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob, multi_class='ovo')}")

2024-09-30 13:16:48.393323: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/gcekcse/hkenv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


ValueError: Expected all entries in the `metrics` list to be metric objects. Received instead:
metrics=[[<MeanMetricWrapper name=accuracy>]]

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
